<a href="https://colab.research.google.com/github/Sriharish19/Cointab_assessment/blob/main/Cointab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


In [2]:
# Mounting the drive
from google.colab import drive
drive.mount('/content/My_drive')


Mounted at /content/My_drive


In [3]:
# Loading the file
df = pd.read_excel('/content/My_drive/MyDrive/Cointab/Company X - Order Report.xlsx')

In [5]:
df.duplicated().sum()
     

2

In [8]:
df1=df.drop_duplicates()


In [11]:
df2=pd.read_excel('/content/My_drive/MyDrive/Cointab/Company X - SKU Master.xlsx')

In [14]:
df2.drop_duplicates(inplace=True)

In [ ]:
df3=pd.merge(df1,df2,how='left',on='SKU')


In [47]:
df3.rename(columns = {'Order Qty':'Order_Qty','Weight (g)':'Weight_g','ExternOrderNo':'Order ID'}, inplace = True)

In [48]:
df3['total_weight']=df3['Order_Qty']*df3['Weight_g']

In [26]:
zone=pd.read_excel('/content/My_drive/MyDrive/Cointab/Company X - Pincode Zones.xlsx')

In [28]:
zone.rename(columns = {'Zone':'Zone1'}, inplace = True)


In [29]:
zone.drop_duplicates(inplace=True)

In [30]:
zone.columns

Index(['Warehouse Pincode', 'Customer Pincode', 'Zone1'], dtype='object')

In [31]:
zone=zone[['Customer Pincode', 'Zone1']]

In [33]:
cc=pd.read_excel('/content/My_drive/MyDrive/Cointab/Courier Company - Invoice.xlsx')

In [37]:
cc1=cc[['Order ID','Customer Pincode','Zone']]

zone_compare=pd.merge(cc1,zone[['Customer Pincode','Zone1']],how='left',on='Customer Pincode')

In [38]:
zone_compare['mismatched']=(zone_compare['Zone'] != zone_compare['Zone1']).astype(int)

In [51]:
company_orders=pd.merge(zone_compare[['Order ID','Zone1']],df3,how='left',on='Order ID')

In [59]:
company_orders1=company_orders[['Order ID','Zone1','Order_Qty','xWeight_g','xtotal_weight']]

In [60]:
company_orders2 = company_orders1.groupby('Order ID').sum()

In [56]:
company_orders.rename(columns={'xOrder ID':'Order ID', 'SKU':'xSKU',  'Weight_g':'xWeight_g', 'total_weight':'xtotal_weight'},inplace=True)

In [ ]:
new=pd.merge(cc,company_orders2,on='Order ID')
new.head(15)

In [68]:
new=pd.merge(new,zone,how='left',on='Customer Pincode')

In [70]:
num_dissimilar = new['Zone'].ne(new['Zone1']).sum()

In [72]:
wz=new[['xtotal_weight','Zone','Type of Shipment']]

In [76]:
new['key']=new['Zone1']+' ' +new['Type of Shipment']

In [81]:
def get_weight_slab(weight):
    if weight <= 500:
        return 0.5
    elif weight <= 1000:
        return 1
    elif weight <= 1500:
        return 1.5
    elif weight <= 2000:
        return 2
    elif weight <= 2500:
        return 2.5
    elif weight <= 3000:
        return 3
    elif weight <= 3500:
        return 3.5

# apply the function to create a new column for weight slab
new['weight_slab'] = new['xtotal_weight'].apply(get_weight_slab)

In [ ]:
new

In [90]:
charges_dict = {
    'a Forward charges': {'fixed': 29.5, 'additional': 23.6},
    'b Forward charges': {'fixed': 33, 'additional': 28.3},
    'c Forward charges': {'fixed': 40.1, 'additional': 38.9},
    'd Forward charges': {'fixed': 45.4, 'additional': 44.8},
    'e Forward charges': {'fixed': 56.6, 'additional': 55.5},
    'a Forward and RTO charges': {'fixed': 43.1, 'additional': 47.2},
    'b Forward and RTO charges': {'fixed': 53.5, 'additional': 53.5},
    'c Forward and RTO charges': {'fixed': 72, 'additional': 72},
    'd Forward and RTO charges': {'fixed': 86.7, 'additional': 86.7},
    'e Forward and RTO charges': {'fixed': 107.3, 'additional': 107.3}
}



In [99]:
def charges(a,b):
  x=0
  if a==0.5:
    x=charges_dict[b]['fixed']
  else:
    x=charges_dict[b]['fixed']+charges_dict[b]['additional']*(int(a/0.5)-1)
  return x


135.0


In [101]:
new['Actual_amount'] = new.apply(lambda x: charges(x['weight_slab'], x['key']) , axis=1)

In [102]:
new

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),Order_Qty,xWeight_g,xtotal_weight,Zone1,key,weight_slab,Actual_amount
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,8.0,377,1302.0,d,d Forward charges,1.5,135.0
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,4.0,615,615.0,d,d Forward charges,1.0,90.2
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,12.0,929,2265.0,d,d Forward charges,2.5,224.6
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,2.0,700,700.0,b,b Forward charges,1.0,61.3
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,1.0,240,240.0,d,d Forward charges,0.5,45.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,90.2,3.0,500,500.0,b,b Forward charges,0.5,33.0
120,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7,3.0,607,607.0,b,b Forward and RTO charges,1.0,107.0
121,1091120922803,2001820978,0.50,121003,313301,d,Forward charges,45.4,2.0,515,515.0,b,b Forward charges,1.0,61.3
122,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,33.0,4.0,689,689.0,e,e Forward charges,1.0,112.1


In [105]:
new1=new[['Billing Amount (Rs.)','Actual_amount']]



,Billing Amount (Rs.),Actual_amount
0,135.0,135.0
1,90.2,90.2
2,224.6,224.6
3,61.3,61.3
4,45.4,45.4
...,...,...
119,90.2,33.0
120,86.7,107.0
121,45.4,61.3
122,33.0,112.1


In [114]:

# define function to compute correct, overcharged, and undercharged orders
def compute_orders(row):
    billing_amount = row['Billing Amount (Rs.)']
    actual_amount = row['Actual_amount']
    
    if billing_amount == actual_amount:
        return ('correct', 1, billing_amount)
    elif actual_amount > billing_amount:
        return ('undercharged', 1, actual_amount - billing_amount)
    else:
        return ('overcharged', 1, billing_amount - actual_amount)

# apply function to create new columns
new1[['order_type', 'order_count', 'total_amount']] = new1.apply(compute_orders, axis=1, result_type='expand')

# groupby to get count and total amount for each order type
order_summary = new1.groupby('order_type').agg({'order_count': 'sum', 'total_amount': 'sum'})

# print order summary
order_summary

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,order_count,total_amount
order_type,,
correct,16,1314.6
overcharged,85,4483.2
undercharged,23,524.2
